In [ ]:
!pwd

/content


In [ ]:
#!/bin/bash
!curl -L -o /content/brazilian-ecommerce.zip\
  https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42.6M  100 42.6M    0     0  65.8M      0 --:--:-- --:--:-- --:--:-- 65.8M


In [ ]:
!unzip /content/brazilian-ecommerce.zip -d /content/brazilian-ecommerce/

Archive:  /content/brazilian-ecommerce.zip
  inflating: /content/brazilian-ecommerce/olist_customers_dataset.csv  
  inflating: /content/brazilian-ecommerce/olist_geolocation_dataset.csv  
  inflating: /content/brazilian-ecommerce/olist_order_items_dataset.csv  
  inflating: /content/brazilian-ecommerce/olist_order_payments_dataset.csv  
  inflating: /content/brazilian-ecommerce/olist_order_reviews_dataset.csv  
  inflating: /content/brazilian-ecommerce/olist_orders_dataset.csv  
  inflating: /content/brazilian-ecommerce/olist_products_dataset.csv  
  inflating: /content/brazilian-ecommerce/olist_sellers_dataset.csv  
  inflating: /content/brazilian-ecommerce/product_category_name_translation.csv  


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("OlisData").getOrCreate()

In [ ]:
spark

In [ ]:
hdfs_path = "/content/brazilian-ecommerce"

In [ ]:
customer_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/olist_customers_dataset.csv")


In [ ]:
customer_df.show(10)

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [ ]:
customer_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/olist_customers_dataset.csv")
geolocation_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/olist_geolocation_dataset.csv")
order_items_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/olist_order_items_dataset.csv")
order_payments_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/olist_order_payments_dataset.csv")
order_reviews_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/olist_order_reviews_dataset.csv")
orders__df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/olist_orders_dataset.csv")
products_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/olist_products_dataset.csv")
sellers_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/olist_sellers_dataset.csv")
product_category_name_translation = spark.read.format("csv").option("header","true").option("inferSchema","true").load(hdfs_path+"/product_category_name_translation.csv")


In [ ]:
geolocation_df.show(5)

+---------------------------+-------------------+------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|   geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+------------------+----------------+-----------------+
|                       1037| -23.54562128115268|-46.63929204800168|       sao paulo|               SP|
|                       1046|-23.546081127035535|-46.64482029837157|       sao paulo|               SP|
|                       1046| -23.54612896641469|-46.64295148361138|       sao paulo|               SP|
|                       1041|  -23.5443921648681|-46.63949930627844|       sao paulo|               SP|
|                       1035|-23.541577961711493|-46.64160722329613|       sao paulo|               SP|
+---------------------------+-------------------+------------------+----------------+-----------------+
only showing top 5 rows


In [ ]:
order_items_df.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18|12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.9|        18.14|
+--------------------+-------------+------------

#Data Lekage or Drop

In [ ]:
print(f'Customers : {customer_df.count()}')
print(f'Geolocation : {geolocation_df.count()}')

Customers : 99441
Geolocation : 1000163


#Check for nulls

In [ ]:
from pyspark.sql.functions import col,isnan,when,count
customer_df.select([col(c).isNull().alias(c) for c in customer_df.columns]).show()


+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|                   false|        false|         false|
|      false|             false|           

In [ ]:
 customer_df.select([count(when(col(c).isNull(),1)).alias(c) for c in customer_df.columns]).show()

+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



#Duplicate values

In [ ]:
customer_df.groupBy("customer_id").count().filter("count > 1").show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
+-----------+-----+



#Customer distribution by state

In [ ]:
customer_df.groupBy("customer_state").agg(count("customer_id").alias("customer_count")).orderBy("customer_count",ascending = False).show()

+--------------+--------------+
|customer_state|customer_count|
+--------------+--------------+
|            SP|         41746|
|            RJ|         12852|
|            MG|         11635|
|            RS|          5466|
|            PR|          5045|
|            SC|          3637|
|            BA|          3380|
|            DF|          2140|
|            ES|          2033|
|            GO|          2020|
|            PE|          1652|
|            CE|          1336|
|            PA|           975|
|            MT|           907|
|            MA|           747|
|            MS|           715|
|            PB|           536|
|            PI|           495|
|            RN|           485|
|            AL|           413|
+--------------+--------------+
only showing top 20 rows


#Order- Order status distribution

In [ ]:
orders__df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [ ]:
orders__df.groupBy("order_status").agg(count("order_id").alias('count_status')).orderBy("count_status",ascending=False).show()

+------------+------------+
|order_status|count_status|
+------------+------------+
|   delivered|       96478|
|     shipped|        1107|
|    canceled|         625|
| unavailable|         609|
|    invoiced|         314|
|  processing|         301|
|     created|           5|
|    approved|           2|
+------------+------------+



#Top selling product

#payments

In [ ]:
order_payments_df.groupBy("payment_type").agg(count("order_id").alias("payment_count")).orderBy("payment_count",ascending=False).show()

+------------+-------------+
|payment_type|payment_count|
+------------+-------------+
| credit_card|        76795|
|      boleto|        19784|
|     voucher|         5775|
|  debit_card|         1529|
| not_defined|            3|
+------------+-------------+

